In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

In [2]:
# Downloaded from Kaggle - https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
df = pd.read_csv('diabetes.csv')

# Let's see the df
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [3]:
# The last column is the Y (target) label, and the other columns are the input variables. Let's split the df into X and Y accordingly.
x = df.iloc[:,:-1].values
y = list(df.iloc[:,-1])

# Let's see this too
print("x:\n", x[:3])
print("y:\n", y[:3])

# Convert the Y label into numeric form
y_int = [int(row == 'positive') for row in y]
print("y_int:\n", y_int[:3])

# Convert the y_int into an np.array
y = np.array(y_int, dtype = 'float64')
print("np y:", y[:3])

x:
 [[  6.  148.   72.   35.    0.   33.6  50. ]
 [  1.   85.   66.   29.    0.   26.6  31. ]
 [  8.  183.   64.    0.    0.   23.3  32. ]]
y:
 ['positive', 'negative', 'positive']
y_int:
 [1, 0, 1]
np y: [1. 0. 1.]


In [4]:
# Now let's normalize the features with the Standard Scaler to get them in the range of (-1, 1)
sc = StandardScaler()
x =  sc.fit_transform(x)

print(x[:3])

[[ 0.63994726  0.84832379  0.14964075  0.90726993 -0.69289057  0.20401277
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575  0.53090156 -0.69289057 -0.68442195
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 -1.28821221 -0.69289057 -1.10325546
  -0.10558415]]


In [5]:
# Now we convert the arrays to PyTorch tensors
x = torch.tensor(x)
# We add an extra dimension to convert this array to 2D
y = torch.tensor(y).unsqueeze(1)

In [6]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [8]:
dataset = Dataset(x,y)

In [9]:
len(dataset)

768

In [10]:
# Load the data to your dataloader for batch processing and shuffling
train_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                           batch_size=32, 
                                           shuffle=True)

In [11]:
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


![network](https://user-images.githubusercontent.com/30661597/60379583-246e5e80-9a68-11e9-8b7f-a4294234c201.png)

In [12]:
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)

        return out

In [13]:
net = Model(x.shape[1])

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

/Users/sthakkar.cw/Documents/Jupyter Notebooks/venv/lib/python3.12/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [14]:
num_epochs = 200

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()

        output = net(inputs)

        loss = criterion(output, labels)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    output = (output > 0.5).float()
    accuracy = (output == labels).float().mean()

    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1, num_epochs, loss, accuracy))

Epoch 1/200, Loss: 0.498, Accuracy: 0.812
Epoch 2/200, Loss: 0.410, Accuracy: 0.844
Epoch 3/200, Loss: 0.492, Accuracy: 0.781
Epoch 4/200, Loss: 0.380, Accuracy: 0.875
Epoch 5/200, Loss: 0.321, Accuracy: 0.875
Epoch 6/200, Loss: 0.437, Accuracy: 0.750
Epoch 7/200, Loss: 0.458, Accuracy: 0.719
Epoch 8/200, Loss: 0.448, Accuracy: 0.750
Epoch 9/200, Loss: 0.350, Accuracy: 0.844
Epoch 10/200, Loss: 0.445, Accuracy: 0.750
Epoch 11/200, Loss: 0.519, Accuracy: 0.781
Epoch 12/200, Loss: 0.440, Accuracy: 0.750
Epoch 13/200, Loss: 0.471, Accuracy: 0.719
Epoch 14/200, Loss: 0.410, Accuracy: 0.781
Epoch 15/200, Loss: 0.410, Accuracy: 0.812
Epoch 16/200, Loss: 0.395, Accuracy: 0.875
Epoch 17/200, Loss: 0.402, Accuracy: 0.812
Epoch 18/200, Loss: 0.668, Accuracy: 0.719
Epoch 19/200, Loss: 0.500, Accuracy: 0.750
Epoch 20/200, Loss: 0.640, Accuracy: 0.625
Epoch 21/200, Loss: 0.312, Accuracy: 0.875
Epoch 22/200, Loss: 0.788, Accuracy: 0.562
Epoch 23/200, Loss: 0.700, Accuracy: 0.656
Epoch 24/200, Loss: 

In [15]:
from captum.attr import LayerConductance, IntegratedGradients, NeuronConductance

/Users/sthakkar.cw/Documents/Jupyter Notebooks/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
input_tensor = torch.tensor([[45, 1, 130, 85, 0, 0, 35.2]], dtype=torch.float32)  # example input
target_class = 0  # Diabetes predicted as class 1

# Choose the layer to analyze
target_layer = net.fc2

# Neuron conductance
neuron_conductance = NeuronConductance(net, target_layer)
contributions = []

for neuron_index in range(target_layer.out_features):
    attr = neuron_conductance.attribute(input_tensor, neuron_index, target=target_class)
    contributions.append(attr.sum().item())

for i, val in enumerate(contributions):
    print(f"Neuron {i} Contribution: {val:.4f}")

Neuron 0 Contribution: 0.0003
Neuron 1 Contribution: 0.0000
Neuron 2 Contribution: -0.0687
Neuron 3 Contribution: -0.5867


In [27]:
from captum.attr import LayerConductance

model = net

# Define LayerConductance with respect to the intermediate layer (fc2)
layer_cond = LayerConductance(model, model.fc2)

# Analyze a single input or multiple inputs
inputs = x[:100]  # shape: [N, 10]

# Attribution of inputs to **entire fc2 layer**
attributions = layer_cond.attribute(inputs, target=None)  # shape: [100, 4]

# Backtrack: Use LayerConductance with a modified submodel to isolate a single neuron
neuron_weights = model.fc2.weight.detach()  # shape [4, 8]
feature_to_neuron = []

for neuron_index in range(model.fc2.out_features):
    # Get gradients for neuron N wrt input
    neuron_attr = LayerConductance(model, model.fc2)
    attr = neuron_attr.attribute(inputs, target=neuron_index)  # shape: [100, 10]
    mean_attr = attr.mean(dim=0).detach().numpy()  # [10]
    feature_to_neuron.append(mean_attr)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Convert to heatmap: rows = neurons, cols = input features
heatmap_data = np.array(feature_to_neuron)  # shape [4, 10]
plt.figure(figsize=(10, 4))
sns.heatmap(heatmap_data, annot=True, cmap="coolwarm", xticklabels=data.feature_names)
plt.xlabel("Input Features")
plt.ylabel("Neuron Index (fc2)")
plt.title("Feature Influence on fc2 Neurons")
plt.show()

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float